generate qivarfiles (equal path weights throughout)

In [5]:
import shutil

homedir = "/home/annzhou"
dirprefix = f"{homedir}/DRing/src/emp/datacentre"
# su2pwfile = f"{dirprefix}/pathweightfiles/dring/su2/pathweight_dring_su2_equal_64.txt"
# su2qivarfile = f"{dirprefix}/qivarfiles/qivar_dring_0_0_su2_64"
# shutil.copy(su2pwfile, su2qivarfile)
# su3pwfile = f"{dirprefix}/pathweightfiles/dring/su3/pathweight_dring_su3_equal_64.txt"
# su3qivarfile = f"{dirprefix}/qivarfiles/qivar_dring_0_0_su3_64"
# shutil.copy(su3pwfile, su3qivarfile)

trafficfilenamearr = ["prv1","cluster_a"]
intervalstartarr = [4,24]
intervalendarr = [8,48]
routingnamearr = ["su3"]
for routingname in routingnamearr:
    for itrafficfilename,trafficfilename in enumerate(trafficfilenamearr):
        intervalstart = intervalstartarr[itrafficfilename]
        intervalend = intervalendarr[itrafficfilename]
        for i in range(intervalstart,intervalend):
            fhifile = f"{dirprefix}/qivarfiles/qivar_dring_0_0_{routingname}_64"
            qivarfile = f"{dirprefix}/qivarfiles/qivar_dring_0_0_{trafficfilename}_{i}_{routingname}_2_0"
            shutil.copy(fhifile, qivarfile)

generate conf files

In [3]:
trafficname = "prv1"
multstrarr1 = ["0 2 100","0 5 100","0 7 100","0 10 100","0 12 100","0 15 100","0 17 100","0 20 100","0 22 100","0 25 100","0 37 100"]
multnamearr1 = list(range(2,19,2))+list(range(20,31,10))
# multstrarr1 = ["0 25 100"]
# multnamearr1 = [20]
sseedarr = range(1,6)

conffile = f"/home/annzhou/DRing/src/emp/datacentre/experiments/controllerfailure2/{trafficname}.conf"
make_leafspine = "MAKE"
make_dring = "MAKE"
with open(conffile,'w') as f:
    for imultstr,multstr in enumerate(multstrarr1):
        for sseed in sseedarr:
            multname = multnamearr1[imultstr]
            # f.write(f"./run.sh LEAFSPINE {make_leafspine} 80 3072 64 leafspine_{trafficname} NEW_WISC null {multstr} 4 8 0 0 0 0 ecmp 0 {trafficname} {sseed} netpathfiles/netpath_ecmp_leafspine.txt qvarfiles/qvar_leafspine_0_0_ecmp_64 64 50 150 200 0 0 > m_leafspine_{trafficname}_{multname}_{sseed}.log\n")
            make_leafspine = "NOMAKE"
            f.write(f"./run.sh RRG {make_dring} 80 2988 64 dring_{trafficname} NEW_WISC graphfiles/ring_supergraph/double_ring/instance1_80_64.edgelist {multstr} 4 8 0 0 0 0 null 0 {trafficname} serverfiles/dring_2988_80_64 {sseed} null null 64 50 150 200 0 0 > m_dring_cf2_{trafficname}_{multname}_{sseed}.log\n")
            make_dring = "NOMAKE"

In [4]:
trafficname = "cluster_a"
multstrarr1 = ["4 8 100","8 17 100","12 25 100","16 34 100","20 42 100","24 51 100","28 59 100","32 67 100","36 76 100"]
multnamearr1 = range(10,91,10)
sseedarr = range(1,6)

conffile = f"/home/annzhou/DRing/src/emp/datacentre/experiments/controllerfailure2/{trafficname}.conf"
make_leafspine = "MAKE"
make_dring = "MAKE"
with open(conffile,'w') as f:
    for imultstr,multstr in enumerate(multstrarr1):
        for sseed in sseedarr:
            multname = multnamearr1[imultstr]
            # f.write(f"./run.sh LEAFSPINE {make_leafspine} 80 3072 64 leafspine_{trafficname} NEW_WISC null {multstr} 24 48 0 0 0 0 ecmp 0 {trafficname} {sseed} netpathfiles/netpath_ecmp_leafspine.txt qvarfiles/qvar_leafspine_0_0_ecmp_64 64 50 150 200 0 0 > m_leafspine_{trafficname}_{multname}_{sseed}.log\n")
            make_leafspine = "NOMAKE"
            f.write(f"./run.sh RRG {make_dring} 80 2988 64 dring_{trafficname} NEW_WISC graphfiles/ring_supergraph/double_ring/instance1_80_64.edgelist {multstr} 24 48 0 0 0 0 null 0 {trafficname} serverfiles/dring_2988_80_64 {sseed} null null 64 50 150 200 0 0 > m_dring_cf2_{trafficname}_{multname}_{sseed}.log\n")
            make_dring = "NOMAKE"

generate sumfile & picklefile

In [ ]:
import pickle

trafficname = "prv1"

homedir = "/u/az6922"
datadict = dict()
conffile = f"{homedir}/DRing/src/emp/datacentre/experiments/controllerfailure2/{trafficname}.conf"
sumfile = f"{homedir}/DRing/src/emp/datacentre/experiments/controllerfailure2/{trafficname}.txt"
with open(conffile,'r') as f:
    with open(sumfile,'a') as sumf:
        lines = f.readlines()
        for line in lines:
            tokens = line.split()
            logfile = tokens[32]
            sizelist = list()
            durationlist = list()
            startlist = list()
            with open(f"{homedir}/DRing/src/emp/datacentre/{logfile}",'r') as logf:
                print(logfile)
                loglines = logf.readlines()
                for logline in loglines:
                    logtokens = logline.split()
                    if logtokens[0] == "FCT":
                        durationlist.append(float(logtokens[2]))
                        sizelist.append(int(logtokens[1]))
                        startlist.append(float(logtokens[3]))
            datadict[logfile] = [durationlist,sizelist,startlist]
            durationlist.sort()
            sumf.write(f"{logfile}\t{sum(durationlist)/len(durationlist)}\t{durationlist[int(len(durationlist)*0.5)]}\t{durationlist[int(len(durationlist)*0.99)]}\t{durationlist[int(len(durationlist)*0.9999)]}\n")

with open(f'{homedir}/DRing/src/emp/datacentre/experiments/controllerfailure2/{trafficname}.pickle', 'wb') as handle:
    pickle.dump(datadict, handle, protocol=pickle.HIGHEST_PROTOCOL)

delete log,run files

In [ ]:
import os

homedir = "/u/az6922"
dringdir = f"{homedir}/DRing/src/emp/datacentre"
for filedir in os.listdir(dringdir):
    if os.path.isfile(os.path.join(dringdir, filedir)):
        tokens = filedir.split("_")
        if tokens[0]=="m" and tokens[1]=="dring" and tokens[2]=="cf2" and tokens[-1].split('.')[-1]=="log":
            print(filedir)
            os.remove(f"{homedir}/DRing/src/emp/datacentre/{filedir}")
        if (tokens[0]=="rrg" or tokens[0]=="leafspine") and tokens[1]=="log":
            print(filedir)
            os.remove(f"{homedir}/DRing/src/emp/datacentre/{filedir}")
        if (tokens[-1]=="run") and ((tokens[0]=="leafspine" and tokens[1]=="leafspine") or (tokens[0]=="rrg" and (tokens[1]=="rrg" or tokens[1]=="dring"))):
            print(filedir)
            os.remove(f"{homedir}/DRing/src/emp/datacentre/{filedir}")